In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as smf

In [101]:
lr_data = pd.read_csv('CDC-2019-2023-DATA_nums.csv', low_memory=False)

In [102]:
lr_data.columns

Index(['Unnamed: 0', 'BIRTHSEX', 'MENTHLTH', 'POORHLTH', 'ADDEPEV3', 'DECIDE',
       'DIFFALON', 'IYEAR', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN',
       'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM',
       'ACEHVSEX', 'EMPLOY1', 'AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH'],
      dtype='object')

In [103]:
lr_data = lr_data.drop(['Unnamed: 0'], axis=1)

In [104]:
lr_data.head()

,BIRTHSEX,MENTHLTH,POORHLTH,ADDEPEV3,DECIDE,DIFFALON,IYEAR,ACEDEPRS,ACEDRINK,ACEDRUGS,...,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,EMPLOY1,AVEDRNK3,EXEROFT1,STRENGTH,PHYSHLTH
0,NaN,0.0,0.0,NaN,No,No,2019,No,No,No,...,More than once,Never,Never,Never,Never,Retired,NaN,NaN,210.0,15.0
1,NaN,0.0,10.0,NaN,No,No,2019,No,Yes,No,...,Once,More than once,Never,Never,Never,Retired,NaN,202.0,0.0,10.0
2,NaN,30.0,0.0,NaN,No,No,2019,No,No,No,...,Never,Never,Never,Never,Never,Retired,NaN,101.0,102.0,0.0
3,NaN,0.0,0.0,NaN,No,NaN,2019,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Retired,NaN,NaN,NaN,30.0
4,NaN,0.0,NaN,NaN,No,No,2019,No,No,No,...,Never,Never,Never,Never,Never,A homemaker,NaN,NaN,0.0,0.0


In [94]:
df2 = lr_data[['MENTHLTH', 'POORHLTH', 'PHYSHLTH','AVEDRNK3', 'STRENGTH']]

In [95]:
corr=df2.corr()
px.imshow(corr, text_auto='.2f', color_continuous_scale='Algae')


In [96]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression


In [97]:
y = lr_data['MENTHLTH']
X = lr_data[['POORHLTH','DECIDE', 'DIFFALON', 'IYEAR', 
        'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS','ACEPRISN', 
        'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR',
        'ACETOUCH','ACETTHEM', 'ACEHVSEX', 'EMPLOY1', 'AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH']]

In [98]:
nums = ['AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH', 'POORHLTH']
cats = ['IYEAR', 'ACEDEPRS', 
        'DECIDE', 'DIFFALON', 'ACEDRINK', 
        'ACEDRUGS','ACEPRISN', 'ACEDIVRC', 
        'ACEPUNCH', 'ACEHURT1', 'ACESWEAR',
        'ACETOUCH','ACETTHEM', 'ACEHVSEX', 'EMPLOY1']

In [99]:
preprocess = ColumnTransformer(transformers=[('encoder',OneHotEncoder(drop='first'),cats),
                                             ('numeric','passthrough',nums)])

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42,stratify=y)

ValueError: Input y contains NaN.

In [ ]:
poly3_model = Pipeline([
    ("preprocess", preprocess),
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("scaler", StandardScaler()),
    ("linreg", LinearRegression())
])


In [ ]:
models = {
    "Polynomial (cubic)": poly3_model,
}


In [ ]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

rows = []
for name, model in models.items():
    model.fit(X_train, y_train)

    yhat_tr = model.predict(X_train)
    yhat_te = model.predict(X_test)

    rows.append({
        "Model": name,
        "Train RMSE": rmse(y_train, yhat_tr),
        "Test RMSE": rmse(y_test, yhat_te),
        "Train R2": r2_score(y_train, yhat_tr),
        "Test R2": r2_score(y_test, yhat_te),
    })

results = pd.DataFrame(rows).sort_values("Test RMSE")
print(results.round(4))

ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values